In [1]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
import matplotlib.pyplot as plt
import re
import scipy
import math
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.fasttext import FastText

2024-07-17 18:30:45.239649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 18:30:45.239790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 18:30:45.410638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd 
data= pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json", lines=True)

In [4]:
data

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [5]:
# Subset data for gensim fastText model
all_text = list(data['text']) # select "text" column only
text= all_text[0:50000] # select first 1000 sample lines
#print("\nSamples of Sentences\n [{}]".format(text[0:10]))

In [6]:
text

['Avengers time with the ladies.',
 'They have lots of good deserts and tasty cuban sandwiches',
 "It's open even when you think it isn't",
 'Very decent fried chicken',
 'Appetizers.. platter special for lunch',
 'Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.',
 "Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!",
 'This is probably the best place in the cool Springs area to watch a game and eat',
 'Tacos',
 'Starbucks substitute in boring downtown Tampa. Ugh. Never again!',
 'Order the Tortilla Soup',
 'Very good will definitely be coming back!!',
 'If the Hotlight is on you must stop in.',
 "Let's go Yankees!",
 "Basically same food as rally's for $5 more",
 "Don't go for dinner. They close at 6. Really Yvonne L? Negative stars for a restaurant where you you didn't and never intended 

In [7]:
def prepocessing(text, stopWords = set(stopwords.words('english')),unique_words_option=True):
    text=text.lower()
    text=text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\w*\d\w*' , ' ' , text)
    text = re.sub(r'[0-9]' , ' ' , text)
    text = re.sub(r'[^a-zA-Z0-9\s]' , ' ' , text)
    text = re.sub(r'\s+' , ' ', text)
    text = word_tokenize(text)
    text = [i for i in text if i != '']
    text = [i for i in text if i not in stopWords]
#     stemmer = PorterStemmer()
#     words = [stemmer.stem(word) for word in text]
    #if unique_words_option: unique_words= set(text)
    #else:return text
    unique_words= set(text)
    unique_words = [word for word in unique_words if len(word) > 3]
    return unique_words

In [8]:
def train_Fasttext(word_tokens,embedding_size,window_size,min_word):
 fast_Text_model = FastText(word_tokens,
 vector_size=embedding_size, # Dimensionality of the word vectors. ,
 window=window_size,
 min_count=min_word, # The model ignores all words with total frequency lower than this.
 #sample=down_sampling, # threshold which higher-frequency words are randomly down sampled
 workers = 4, # Num threads to train the model (faster training with multicore comp.)
 sg=1, # Training algorithm: skip-gram if sg=1, otherwise CBOW.
 epochs=100) # Number of iterations (epochs) over the corpus
 fast_Text_model.save('fasttext_model')

In [9]:
sentences_as_lists = [sentence.split(',') for sentence in text]
input_text=[]
for i in sentences_as_lists:
  input_text.append(prepocessing(str(i)))


# Printing the result
print(input_text)

[['ladies', 'avengers', 'time'], ['deserts', 'good', 'tasty', 'sandwiches', 'cuban', 'lots'], ['open', 'think', 'isnt', 'even'], ['decent', 'chicken', 'fried'], ['lunch', 'appetizers', 'special', 'platter'], ['vanilla', 'pickle', 'onion', 'cheeseburger', 'chili', 'single', 'relish', 'cocacolaso'], ['stops', 'patcos', 'leave', 'trains', 'ride', 'saturday', 'great', 'city', 'center', 'kids', 'train', 'lindenwold', 'elves', 'silver', 'sleigh', 'santa', 'decorated', 'make'], ['game', 'watch', 'springs', 'cool', 'best', 'place', 'probably', 'area'], ['tacos'], ['downtown', 'substitute', 'starbucks', 'tampa', 'boring', 'never'], ['soup', 'order', 'tortilla'], ['back', 'good', 'definitely', 'coming'], ['hotlight', 'stop', 'must'], ['yankees', 'lets'], ['basically', 'rallys', 'food'], ['intended', 'close', 'stars', 'dinner', 'yvonne', 'negative', 'dont', 'really', 'didnt', 'restaurant', 'never'], ['tight', 'take', 'mins', 'right'], ['environment', 'delicious', 'entree', 'though', 'chicken', 'l

In [10]:
train_Fasttext(input_text,300,5,1)

In [11]:
model = FastText.load('fasttext_model')

In [12]:
test=np.random.choice(prepocessing(str(text)),20, replace=False)
test

array(['artown', 'fenomenal', 'preferred', 'fooooddd', 'philip',
       'declining', 'cupholders', 'hobbytoy', 'surfin', 'solved',
       'nsteer', 'sadness', 'phones', 'zaalld', 'terriyaki', 'locationto',
       'refunded', 'rowngot', 'freshness', 'says'], dtype='<U128')

In [13]:
print(model.wv.most_similar(test[0], topn=10))

[('ttown', 0.7798545956611633), ('htown', 0.7630123496055603), ('etown', 0.7570963501930237), ('janktown', 0.7321745157241821), ('laketown', 0.7240025997161865), ('naptown', 0.7092666029930115), ('dogtown', 0.7077212929725647), ('motown', 0.7022055387496948), ('ntown', 0.6992093920707703), ('havertown', 0.6972903609275818)]


In [14]:
print(model.wv.most_similar(negative=[test[0]],topn=10))

[('need', -0.0858604684472084), ('slower', -0.08908454328775406), ('hate', -0.09068664163351059), ('getting', -0.09181052446365356), ('clear', -0.09503237903118134), ('ginger', -0.09645480662584305), ('scooter', -0.09998709708452225), ('friend', -0.10128927230834961), ('pancakes', -0.10274121910333633), ('immediately', -0.10519726574420929)]


In [15]:
for i in test:
  print(i," ",model.wv.most_similar(i,topn=10))

artown   [('ttown', 0.7798545956611633), ('htown', 0.7630123496055603), ('etown', 0.7570963501930237), ('janktown', 0.7321745157241821), ('laketown', 0.7240025997161865), ('naptown', 0.7092666029930115), ('dogtown', 0.7077212929725647), ('motown', 0.7022055387496948), ('ntown', 0.6992093920707703), ('havertown', 0.6972903609275818)]
fenomenal   [('phenomenal', 0.934553325176239), ('tatphenomenal', 0.9246824979782104), ('phenomenally', 0.8621407747268677), ('menomena', 0.8603017330169678), ('specialsphenomenal', 0.8595093488693237), ('phenomenonal', 0.8479962348937988), ('phenom', 0.7685952186584473), ('phenominal', 0.7119302749633789), ('arsenal', 0.541483998298645), ('promenade', 0.5255275368690491)]
preferred   [('referred', 0.8589507341384888), ('deferred', 0.828122615814209), ('prefers', 0.7321559190750122), ('prefer', 0.7133094668388367), ('referral', 0.6693241596221924), ('prefernces', 0.6636086702346802), ('prefixe', 0.6352576613426208), ('ferrets', 0.6338627934455872), ('stirre

In [16]:
for i in test:
  print(i," ",model.wv.most_similar(negative=[i],topn=10))

artown   [('need', -0.0858604684472084), ('slower', -0.08908454328775406), ('hate', -0.09068664163351059), ('getting', -0.09181052446365356), ('clear', -0.09503237903118134), ('ginger', -0.09645480662584305), ('scooter', -0.09998709708452225), ('friend', -0.10128927230834961), ('pancakes', -0.10274121910333633), ('immediately', -0.10519726574420929)]
fenomenal   [('peopl', -0.04664904624223709), ('community', -0.054997824132442474), ('business', -0.06352107971906662), ('drama', -0.07575500011444092), ('grocery', -0.08089308440685272), ('sometimes', -0.08111417293548584), ('door', -0.08586148917675018), ('stomach', -0.08649680018424988), ('people', -0.08657936751842499), ('seen', -0.08693977445363998)]
preferred   [('risotto', -0.05282606557011604), ('round', -0.05749162659049034), ('generous', -0.06218778342008591), ('kale', -0.06363563239574432), ('snack', -0.0650034099817276), ('sorry', -0.06512493640184402), ('sunset', -0.06857369095087051), ('cure', -0.07163660228252411), ('couldnt

In [17]:
!pip install fasttext

In [18]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-et-vectors", filename="model.bin")

model.bin:   0%|          | 0.00/7.24G [00:00<?, ?B/s]

In [19]:
pre_model = fasttext.load_model(model_path)

In [20]:
pre_model.get_nearest_neighbors(test[0],k=10)

[(0.9442448019981384, 'Baytown'),
 (0.8713819980621338, 'Quakertown'),
 (0.8700230121612549, 'Jumptown'),
 (0.8324531316757202, 'Beardstown'),
 (0.8271458148956299, 'Clovertown'),
 (0.8190239071846008, 'Whiskeytown'),
 (0.8179944753646851, 'Macetown'),
 (0.8165665864944458, 'Bytown'),
 (0.8154719471931458, 'Ivytown'),
 (0.8097839951515198, 'Riepetown')]

In [21]:
pre_model.get_nearest_neighbors(test[0], k=len(pre_model.words))[-10:-1]

[(-0.5742716789245605, '東萊區'),
 (-0.5760889649391174, 'Giba'),
 (-0.5781412720680237, '.p'),
 (-0.5781623125076294, 'Õr'),
 (-0.5786856412887573, 'Kf'),
 (-0.5806108117103577, '西區'),
 (-0.5808415412902832, 'Öp'),
 (-0.586741030216217, 'M.T'),
 (-0.5887856483459473, '2C-D')]

In [22]:
for i in test:
    print(i," ",pre_model.get_nearest_neighbors(i,k=10))

artown   [(0.9442448019981384, 'Baytown'), (0.8713819980621338, 'Quakertown'), (0.8700230121612549, 'Jumptown'), (0.8324531316757202, 'Beardstown'), (0.8271458148956299, 'Clovertown'), (0.8190239071846008, 'Whiskeytown'), (0.8179944753646851, 'Macetown'), (0.8165665864944458, 'Bytown'), (0.8154719471931458, 'Ivytown'), (0.8097839951515198, 'Riepetown')]
fenomenal   [(0.88459712266922, 'pHenomenal'), (0.822505533695221, 'phenomenal'), (0.8123601675033569, 'Epiphenomenal'), (0.7840612530708313, 'Epifenomen'), (0.7824857831001282, 'fenomenalism'), (0.774799644947052, 'Phenomenal'), (0.752461314201355, 'fenomenaalsus'), (0.7459837198257446, 'fenomene'), (0.7412140965461731, 'fenomenis'), (0.7409778237342834, 'algfenomen')]
preferred   [(0.8592146039009094, 'prefer'), (0.8439748287200928, 'prefers'), (0.8403026461601257, 'preferably'), (0.831749677658081, 'preferable'), (0.7712801694869995, 'prefetch'), (0.7663486003875732, 'referred'), (0.7505120038986206, 'prefixes'), (0.7457796931266785,

In [23]:
for i in test:
    print(i," ",pre_model.get_nearest_neighbors(i, k=len(pre_model.words))[-10:-1])

artown   [(-0.5742716789245605, '東萊區'), (-0.5760889649391174, 'Giba'), (-0.5781412720680237, '.p'), (-0.5781623125076294, 'Õr'), (-0.5786856412887573, 'Kf'), (-0.5806108117103577, '西區'), (-0.5808415412902832, 'Öp'), (-0.586741030216217, 'M.T'), (-0.5887856483459473, '2C-D')]
fenomenal   [(-0.5295442938804626, 'pop8'), (-0.5303306579589844, 'HA-'), (-0.5303599238395691, 'v.666'), (-0.5319991111755371, 'cleb'), (-0.5322276949882507, 'Să'), (-0.5332946181297302, 'Vsun'), (-0.5337064266204834, 'žw'), (-0.5379080176353455, 'Irea'), (-0.5387583374977112, '5-MeO-DMT')]
preferred   [(-0.4409538507461548, 'a-Ha'), (-0.44178614020347595, 'T.U'), (-0.44210827350616455, 'Olg'), (-0.4426693916320801, '2C-C'), (-0.4466851055622101, 'Ürge'), (-0.4474621117115021, 'Hõ'), (-0.45317375659942627, 'Kag'), (-0.4536984860897064, 'Kož'), (-0.45403438806533813, 'Giba')]
fooooddd   [(-0.6709496378898621, 'Z-Ż'), (-0.6709865927696228, 'TERG'), (-0.671926736831665, 'šw'), (-0.6726254224777222, 'SiSb'), (-0.67298